<a href="https://colab.research.google.com/github/inEvolving/Portfolio/blob/main/Laboratorio_1_Big_Data_Libertini_Franzoni_Luque.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTAMOS LIBRERIAS A UTILIZAR

In [26]:
import os
import pandas as pd
import matplotlib.pyplot as plt

CREAMOS LA RUTA DE ACCESO A LOS DATASETS

In [27]:
ruta = '/content/drive/MyDrive/UPSO ML BD/DatasetVentas2'

archivos = os.listdir(ruta)
archivos

['Dataset_de_ventas_Enero.csv',
 'Dataset_de_ventas_Abril.csv',
 'Dataset_de_ventas_Agosto.csv',
 'Dataset_de_ventas_Julio.csv',
 'Dataset_de_ventas_Febrero.csv',
 'Dataset_de_ventas_Mayo.csv',
 'Dataset_de_ventas_Junio.csv',
 'Dataset_de_ventas_Marzo.csv',
 'Dataset_de_ventas_Noviembre.csv',
 'Dataset_de_ventas_Septiembre.csv',
 'Dataset_de_ventas_Octubre.csv',
 'Dataset_de_ventas_Diciembre.csv']

MONTAMOS EL DRIVE

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


CREAMOS UN DICCIONARIO CON LA INFORMACION DE LOS MESES (Key=Meses, Values=Valores)
LIMPIAMOS VALORES NULL, Y CASTEAMOS DATOS


In [29]:
dataframe = {}

for archivo in archivos:
  if archivo.endswith('.csv'): #si termina en .csv
    mes = archivo.split('_')[3].split('.')[0] # divide el nombre del archivo
    rutaCompleta = os.path.join(ruta, archivo) #para que funcione la ruta mas el nombre
    df = pd.read_csv(rutaCompleta) #guardo un df el ultimo
    df_ventas = df.copy() #hago copia para no modificar el original

    dataframe[mes] = df_ventas #guardo en el diccionario con el nombre del df

    # Transformo todos los valores no numericos en Nan en las columnas Cantidad Pedida y Precio Unitario
    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors='coerce')  #El errors= 'coerce' transforma los valores no numericos a NaN'
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors='coerce')

    #creo una mascara booleana donde elimina si hay algun valor nulo en las filas
    # elimino todos los Nan y me quedo solo con los valores numericos
    df_ventas = df_ventas.loc[~df_ventas['Cantidad Pedida'].isna()]
    df_ventas = df_ventas.loc[~df_ventas['Precio Unitario'].isna()]

    df_ventas = df_ventas.dropna()

    #cambio el tipo de dato de cada columna
    df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int)
    df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(float)



Creamos la funcion extraer datos que cambia la columna FECHA DE PEDIDO, y la separamos en datetime.


In [30]:
def extraer_datos(datos):
  datos['Fecha de Pedido'] = pd.to_datetime(df_ventas['Fecha de Pedido'])
  datos['Hora'] = datos['Fecha de Pedido'].dt.hour
  datos['Dia'] = datos['Fecha de Pedido'].dt.day
  datos['Nro Mes'] = datos['Fecha de Pedido'].dt.month
  return datos



Creamos el dataframe PRINCIPAL, y dividimos la columna 'Fecha de Pedido' en varias columnas separadas, tambien limpiamos.

In [31]:
dataframe_con_meses = []

for archivo in archivos:
  if archivo.endswith('.csv'):
    ruta_completa = os.path.join(ruta, archivo)
    df = pd.read_csv(ruta_completa)
    mes = archivo.split('_')[3].split('.')[0]
    dataframe_ventas = df.copy()
    dataframe_ventas['Mes'] = mes

    dataframe_con_meses.append(dataframe_ventas)

dataframe_completo = pd.concat(dataframe_con_meses, ignore_index=True)


# Transformo todos los valores no numericos en Nan en las columnas Cantidad Pedida y Precio Unitario
dataframe_completo['Cantidad Pedida'] = pd.to_numeric(dataframe_completo['Cantidad Pedida'], errors='coerce')  #El errors= 'coerce' transforma los valores no numericos a NaN'
dataframe_completo['Precio Unitario'] = pd.to_numeric(dataframe_completo['Precio Unitario'], errors='coerce')

#creo una mascara booleana donde elimina si hay algun valor nulo en las filas
# elimino todos los Nan y me quedo solo con los valores numericos
dataframe_completo = dataframe_completo.loc[~dataframe_completo['Cantidad Pedida'].isna()]
dataframe_completo = dataframe_completo.loc[~dataframe_completo['Precio Unitario'].isna()]

dataframe_completo = dataframe_completo.dropna()

#cambio el tipo de dato de cada columna
dataframe_completo['Cantidad Pedida'] = dataframe_completo['Cantidad Pedida'].astype(int)
dataframe_completo['Precio Unitario'] = dataframe_completo['Precio Unitario'].astype(float)

dataframe_completo['Fecha de Pedido'] = pd.to_datetime(dataframe_completo['Fecha de Pedido'])
dataframe_completo['Hora'] = dataframe_completo['Fecha de Pedido'].dt.hour
dataframe_completo['Dia'] = dataframe_completo['Fecha de Pedido'].dt.day
dataframe_completo['Nro Mes'] = dataframe_completo['Fecha de Pedido'].dt.month



Creamos una nueva columna para calcular el 'Total Generado'


In [32]:
# genero la columna total generado
dataframe_completo['Total Generado'] = dataframe_completo['Cantidad Pedida'] * dataframe_completo['Precio Unitario']
dataframe_completo


,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Mes,Hora,Dia,Nro Mes,Total Generado
0,141234,iPhone,1,700.00,2019-01-22 21:25:00,"944 Walnut St, Boston, MA 02215",Enero,21,22,1,700.00
1,141235,Lightning Charging Cable,1,14.95,2019-01-28 14:15:00,"185 Maple St, Portland, OR 97035",Enero,14,28,1,14.95
2,141236,Wired Headphones,2,11.99,2019-01-17 13:33:00,"538 Adams St, San Francisco, CA 94016",Enero,13,17,1,23.98
3,141237,27in FHD Monitor,1,149.99,2019-01-05 20:33:00,"738 10th St, Los Angeles, CA 90001",Enero,20,5,1,149.99
4,141238,Wired Headphones,1,11.99,2019-01-25 11:59:00,"387 10th St, Austin, TX 73301",Enero,11,25,1,11.99
...,...,...,...,...,...,...,...,...,...,...,...
186845,319666,Lightning Charging Cable,1,14.95,2019-12-11 20:58:00,"14 Madison St, San Francisco, CA 94016",Diciembre,20,11,12,14.95
186846,319667,AA Batteries (4-pack),2,3.84,2019-12-01 12:01:00,"549 Willow St, Los Angeles, CA 90001",Diciembre,12,1,12,7.68
186847,319668,Vareebadd Phone,1,400.00,2019-12-09 06:43:00,"273 Wilson St, Seattle, WA 98101",Diciembre,6,9,12,400.00
186848,319669,Wired Headphones,1,11.99,2019-12-03 10:39:00,"778 River St, Dallas, TX 75001",Diciembre,10,3,12,11.99


Calculamos el Ingreso total generado por mes y ordenamos.




In [33]:
totalXmes = dataframe_completo.groupby('Mes')['Total Generado'].sum()
totalXmesOrdenado = totalXmes.sort_values(ascending=False)
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

totalXmesOrdenado

Mes
Diciembre     4619297.12
Octubre       3736884.05
Abril         3396059.11
Noviembre     3198909.23
Mayo          3144584.80
Marzo         2809063.30
Julio         2646899.69
Junio         2578293.30
Agosto        2244412.31
Febrero       2203481.24
Septiembre    2098816.70
Enero         1815335.12
Name: Total Generado, dtype: float64

Creamos el grafico que muestra el ingreso total generado por mes.

In [34]:
import plotly.graph_objects as go

# Ordenar la serie totalXmes según el orden de la lista meses
totalXmesOrdenadoPorMes = totalXmes.reindex(meses)

# Crear la figura del gráfico de barras
fig = go.Figure()

# Agregar la barra al gráfico
fig.add_trace(go.Bar(
    x=totalXmesOrdenadoPorMes.index,  # Meses en el eje X
    y=totalXmesOrdenadoPorMes.values,  # Totales en el eje Y
    #marker_color='blue',  # Color de las barras

    marker = dict(color= totalXmesOrdenadoPorMes.values, colorscale = 'RdYlGn',colorbar = dict(title = "Ventas Mensuales")),
    text = totalXmesOrdenadoPorMes.values,
    textposition = 'auto'
))

# Agregar título y etiquetas de ejes
fig.update_layout(
    title='Total Generado por Mes',
    xaxis=dict(title='Meses'),
    yaxis=dict(title='Total Generado'),
)

# Mostrar el gráfico
fig.show()

Graficamos las unidades vendidas por mes a lo largo del anio.

In [36]:
import plotly.graph_objects as go

meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']


totales_ventas = []
nombres_meses = []

for mes in meses:
  dataframe_mes = dataframe[mes]
  dataframes_mes = extraer_datos(dataframe_mes)

  total_ventas_mes = dataframe_mes['Cantidad Pedida'].sum()
  totales_ventas.append(total_ventas_mes)
  nombres_meses.append(mes)



fig = go.Figure()

# Agrego al objeto figura un grafico de barra conn los valores extraidos previamente
fig.add_trace(go.Bar(
        x= nombres_meses,
        y=  totales_ventas,
        yaxis="y",
        name="Unidades vendidas",
        marker = dict(color= totales_ventas, colorscale = 'RdYlGn',colorbar = dict(title = "Total de ventas")),
        text = totales_ventas,
        textposition = 'auto'
))

# Configuro titulos y ejes del grafico
fig.update_layout(
    title="Unidades vendidas por mes",
    xaxis=dict(title="Meses"),
    yaxis=dict(title="Unidades vendidas", side="left"))


# Muestro el grafico
fig.show()




¿Cuál es el momento ideal del día para presentar la publicidad y aumentar la
probabilidad de compra?


Graficamos las ventas por hora para cada mes en un solo grafico para poder comparar.

In [38]:
ventasXhora_por_mes = dataframe_completo.groupby(['Mes', 'Hora'])['Cantidad Pedida'].sum()

Horas= list(range(24))
# Crear el objeto figura
fig = go.Figure()

for mes in meses:
    ventas_mes_actual = ventasXhora_por_mes[mes]

    # Agregar al objeto figura un gráfico de línea con las horas y las ventas por hora para el mes actual
    fig.add_trace(go.Scatter(
        x=ventas_mes_actual.index,
        y=ventas_mes_actual.values,
        mode='lines',
        name=mes,
        line=dict(shape='spline', smoothing=1.3, width=2)))

# Configurar los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora",
    xaxis=dict(title="Horas",tickvals=Horas),
    yaxis=dict(title="Unidades vendidas", side="left"))

fig.show()

Creamos una funcion para dividir la columna Direccion de Envio y poder separarlas en 4 mas, para poder trabajar en un futuro con ellas.

In [40]:
def dividir_direccion(datos):
  datos_divididos = datos['Dirección de Envio'].str.split(',')
  datos['Domicilio'] = datos_divididos.str[0]
  datos['Ciudad'] = datos_divididos.str[1]
  datos['Estado'] = datos_divididos.str[2].str.split(' ').str[1]
  datos['Cod Postal'] = datos_divididos.str[2].str.split(' ').str[2]
  return datos

dataframe_con_direcciones = dividir_direccion(dataframe_completo)
dataframe_con_direcciones

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Mes,Hora,Dia,Nro Mes,Total Generado,Domicilio,Ciudad,Estado,Cod Postal
0,141234,iPhone,1,700.00,2019-01-22 21:25:00,"944 Walnut St, Boston, MA 02215",Enero,21,22,1,700.00,944 Walnut St,Boston,MA,02215
1,141235,Lightning Charging Cable,1,14.95,2019-01-28 14:15:00,"185 Maple St, Portland, OR 97035",Enero,14,28,1,14.95,185 Maple St,Portland,OR,97035
2,141236,Wired Headphones,2,11.99,2019-01-17 13:33:00,"538 Adams St, San Francisco, CA 94016",Enero,13,17,1,23.98,538 Adams St,San Francisco,CA,94016
3,141237,27in FHD Monitor,1,149.99,2019-01-05 20:33:00,"738 10th St, Los Angeles, CA 90001",Enero,20,5,1,149.99,738 10th St,Los Angeles,CA,90001
4,141238,Wired Headphones,1,11.99,2019-01-25 11:59:00,"387 10th St, Austin, TX 73301",Enero,11,25,1,11.99,387 10th St,Austin,TX,73301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186845,319666,Lightning Charging Cable,1,14.95,2019-12-11 20:58:00,"14 Madison St, San Francisco, CA 94016",Diciembre,20,11,12,14.95,14 Madison St,San Francisco,CA,94016
186846,319667,AA Batteries (4-pack),2,3.84,2019-12-01 12:01:00,"549 Willow St, Los Angeles, CA 90001",Diciembre,12,1,12,7.68,549 Willow St,Los Angeles,CA,90001
186847,319668,Vareebadd Phone,1,400.00,2019-12-09 06:43:00,"273 Wilson St, Seattle, WA 98101",Diciembre,6,9,12,400.00,273 Wilson St,Seattle,WA,98101
186848,319669,Wired Headphones,1,11.99,2019-12-03 10:39:00,"778 River St, Dallas, TX 75001",Diciembre,10,3,12,11.99,778 River St,Dallas,TX,75001


¿En qué ciudades se han registrado las mayores ventas?


In [51]:
#Ordenado alfabeticamente por Ciudad
ciudades_ventas = dataframe_con_direcciones.groupby('Ciudad')['Total Generado'].sum().round(2)


Ciudad
 Atlanta          2795498.58
 Austin           1819581.75
 Boston           3661642.01
 Dallas           2767975.40
 Los Angeles      5452570.80
 New York City    4664317.43
 Portland         2320490.61
 San Francisco    8262203.91
 Seattle          2747755.48
Name: Total Generado, dtype: float64

Graficamos las cantidades de totales generados por ciudad.

In [42]:
import plotly.graph_objects as go

fig = go.Figure()

# Agrego al objeto figura un grafico de barra conn los valores extraidos previamente
fig.add_trace(go.Bar(
        x= ciudades_ventas.index,
        y= ciudades_ventas.values.round(2),
        yaxis="y",
        name="Mayores Ventas x Ciudad",
        marker = dict(color= ciudades_ventas, colorscale = 'RdYlGn',colorbar = dict(title = "Mayores ventas x ciudad")),
        text = ciudades_ventas,
        textposition = 'auto'
))

# Configuro titulos y ejes del grafico
fig.update_layout(
    title="Mayores Ventas x Ciudad",
    title_x=0.5,
    xaxis=dict(title="Ciudades"),
    yaxis=dict(title="Ventas x Ciudad", side="left"))


# Muestro el grafico
fig.show()


● ¿Cómo se comparan las ventas en diferentes estados o regiones?

In [53]:
estados_ventas = dataframe_con_direcciones.groupby('Estado')['Total Generado'].sum().round(2)

import plotly.graph_objects as go

fig = go.Figure()

# Agrego al objeto figura un grafico de barra conn los valores extraidos previamente
fig.add_trace(go.Bar(
        x= estados_ventas.index,
        y= estados_ventas.values.round(2),
        yaxis="y",
        name="Mayores Ventas x Estado",
        marker = dict(color= estados_ventas, colorscale = 'RdYlGn',colorbar = dict(title = "Totales generados x Estados")),
        text = estados_ventas,
        textposition = 'outside',

))

# Configuro titulos y ejes del grafico
fig.update_layout(
    title="Totales generados x Estados",
    title_x=0.5,
    xaxis=dict(title="Estados"),
    yaxis=dict(title="Totales generados x Estado", side="left"))


# Muestro el grafico
fig.show()

¿Existe variación en las ventas por estado a lo largo de los meses?


Agrupamos los totales generados por mes para cada estado.

In [44]:
ventas_estado_mes = dataframe_completo.groupby(['Estado', 'Mes'])['Total Generado'].sum().round(2)
#ventas_estado_mes_actual = ventas_estado_mes[mes]
#ventas_estado_mes
lisEstados = dataframe_completo['Estado'].unique()
listaEstados = list(lisEstados)
ventas_estado_mes

Estado  Mes       
CA      Abril         1368473.84
        Agosto         884214.79
        Diciembre     1792358.74
        Enero          720717.31
        Febrero        890806.67
                         ...    
WA      Marzo          219512.53
        Mayo           211504.29
        Noviembre      251699.34
        Octubre        317263.99
        Septiembre     156354.32
Name: Total Generado, Length: 96, dtype: float64

Para poder visualizar los datos anteriores de una mejor manera, usamos el metodo unstack().

In [45]:
ventasESTADO = ventas_estado_mes.reset_index()
#ventasESTADO
vent = ventas_estado_mes.unstack()
ventOrdenado = vent[meses]
ventOrdenado

Mes,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
Estado,,,,,,,,,,,,
CA,720717.31,890806.67,1122139.64,1368473.84,1272016.65,1063861.66,1037837.30,884214.79,818748.07,1478269.70,1265330.34,1792358.74
GA,149163.38,176566.45,231811.37,284454.92,239262.96,219395.51,211766.47,169267.66,171278.89,306642.99,275378.71,360509.27
MA,199776.55,215796.39,301063.55,353880.16,327687.88,254859.87,291535.16,240275.25,247368.73,367328.34,351244.07,510826.06
ME,22708.80,29845.49,30516.29,42536.49,57978.76,30025.33,33021.14,35396.60,28759.56,52322.52,34681.22,51966.07
NY,259250.67,305396.20,370198.22,446587.78,436418.35,323874.60,355623.79,303157.81,301290.77,486401.39,428480.34,647637.51
OR,92123.77,119999.35,156298.74,198591.62,172591.21,139553.91,143990.65,116893.09,104189.92,201850.29,172752.60,251897.19
TX,231052.58,293740.44,377522.96,425524.06,427124.70,332868.90,360611.26,305477.07,270826.44,526804.83,419342.61,616661.30
WA,140542.06,171330.25,219512.53,276010.24,211504.29,213853.52,212513.92,189730.04,156354.32,317263.99,251699.34,387440.98


Graficamos lo anteriormente preparado.

In [54]:
# Crear el objeto figura
fig = go.Figure()

for estado in dataframe_completo['Estado'].unique():
    ventas_estado_mes_actual = ventas_estado_mes[estado].reindex(meses)

    # Agregar al objeto figura un gráfico de línea con las horas y las ventas por hora para el mes actual
    fig.add_trace(go.Scatter(
        x=ventas_estado_mes_actual.index,
        y=ventas_estado_mes_actual.values,
        mode='lines',
        name=estado,
        line=dict(shape='spline', smoothing=1.3, width=2)))

# Configurar los ejes y los títulos
fig.update_layout(
    title="Totales Generados por Mes para cada Estado",
    title_x=0.5,
    xaxis=dict(title="Meses",tickvals=list(range(len(meses))), ticktext=meses),
    yaxis=dict(title="Totales Generados", side="left"))

fig.show()

● ¿Cuál es el producto más vendido en general y en cada mes?

Averiguamos el producto mas vendido.

In [48]:
productos_mas_vendidos_delData = dataframe_completo.groupby('Producto')['Cantidad Pedida'].sum()

#busco la ubicacion del mayor y la cantidad maxima
productoMasVendido = productos_mas_vendidos_delData.idxmax()
productoMayorCantidad = productos_mas_vendidos_delData.max()

elMasVendido = dataframe_completo.loc[dataframe_completo['Producto'] == productoMasVendido].max()
print('El producto mas vendido:')
print(elMasVendido['Producto'])
print('La mayor cantidad pedida es:', productoMayorCantidad)


El producto mas vendido:
AAA Batteries (4-pack)
La mayor cantidad pedida es: 31017


En cada mes....


In [49]:
productosMasVendidosXmes = {}

#recorro el dataframe por la columna mes en los meses del un valor unico asi me dan los 12
for mes in dataframe_completo['Mes'].unique():
  mesActual = dataframe_completo[dataframe_completo['Mes'] == mes] #me posiciono en el mes
  ventasMesActual = mesActual.groupby('Producto')['Cantidad Pedida'].sum() #agrupo por

  productoMasVendido = ventasMesActual.idxmax()
  productoMayorCantidad = ventasMesActual.max()

  productosMasVendidosXmes[mes] = {
      'Producto': productoMasVendido,
      'Cantidad': productoMayorCantidad
      }

proMasVen = pd.DataFrame(productosMasVendidosXmes)
#proOrdenados = proMasVen.reindex(meses, axis=1)
proOrdenados = proMasVen[meses]
proOrdenados


,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
Producto,AAA Batteries (4-pack),AAA Batteries (4-pack),AAA Batteries (4-pack),AAA Batteries (4-pack),AAA Batteries (4-pack),AAA Batteries (4-pack),AAA Batteries (4-pack),AAA Batteries (4-pack),AAA Batteries (4-pack),AAA Batteries (4-pack),AAA Batteries (4-pack),AAA Batteries (4-pack)
Cantidad,1596,1975,2478,2936,2931,2160,2366,2019,1927,3362,3027,4240


FORMATO DE GRAFICO DE TORTA



In [ ]:

# Supongamos que tienes el DataFrame díasmasDF con los datos ya preparados

# Crear un nuevo DataFrame para el gráfico de pie
data_pie = pd.DataFrame({'Fecha': díasmasDF.index, 'Total Generado': díasmasDF['Total Generado']})

# Crear el gráfico de pie
fig_pie = px.pie(data_pie, names='Fecha', values='Total Generado', title='Total Generado por Fecha')
fig_pie.show()

● ¿Qué factores crees que han contribuido al éxito de ese producto en particular?
(Se contesta en el informe)

Encontramos el producto mas economico para poder sacar conclusiones con respecto a lo anteriormente encontrado.

In [55]:
#Es el producto mas economico
ubicacionProductoMasBarato = dataframe_con_direcciones['Precio Unitario'].idxmin()
productoMasBaratoValor = dataframe_con_direcciones['Precio Unitario'].min()
producto_mas_economico = dataframe_con_direcciones.loc[dataframe_con_direcciones['Precio Unitario'] == productoMasBaratoValor]

producto_mas_economico.min()

ID de Pedido                              141239
Producto                  AAA Batteries (4-pack)
Cantidad Pedida                                1
Precio Unitario                             2.99
Fecha de Pedido              2019-01-01 05:53:00
Dirección de Envio    1 4th St, Dallas, TX 75001
Mes                                        Abril
Hora                                           0
Dia                                            1
Nro Mes                                        1
Total Generado                              2.99
Domicilio                               1 4th St
Ciudad                                   Atlanta
Estado                                        CA
Cod Postal                                 02215
dtype: object

Mostramos el producto mas vendido y su cantidad.


In [56]:
#Agrupar los datos por producto y sumar las cantidades vendidas
ventas_por_producto = dataframe_completo.groupby('Producto')['Cantidad Pedida'].sum()

# Encontrar el producto más vendido en general
producto_mas_vendido = ventas_por_producto.idxmax()
cantidad_maxima_vendida = ventas_por_producto.max()

# Paso 3: Encontrar la fila correspondiente en el DataFrame original
producto_mas_vendido_fila = dataframe_completo.loc[dataframe_completo['Producto'] == producto_mas_vendido]

# Paso 4: Mostrar los resultados
print("Producto más vendido en general:", producto_mas_vendido)
print("Cantidad máxima vendida:", cantidad_maxima_vendida)





Producto más vendido en general: AAA Batteries (4-pack)
Cantidad máxima vendida: 31017


Tendencia ventas:

¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días
en los diferentes meses?

In [60]:

#para totales generados
patron = dataframe_con_direcciones.groupby(['Mes','Dia'])['Total Generado'].sum().unstack().reindex(meses)
#patronOrdenado = patron[meses]

#Convertimos en listas para poder graficar luego
listaDias = list(patron.columns)
listaMeses = list(patron.index)
listaValores = list(patron.values)




print(listaDias)
print(listaMeses)
print(listaValores[0])


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
[67430.62 70813.2  47046.2  62012.21 46524.63 52777.49 53676.42 56112.47
 55153.13 56660.92 78414.54 48126.41 61284.39 50090.48 64869.32 51305.
 55152.75 48469.74 56964.73 68084.88 60948.29 59380.92 57572.45 55848.86
 57721.49 71476.18 63683.97 57419.48 61692.21 56238.61 62383.13]


Graficamos para poder encontrar algun patron de ventas.
EN DOS GRAFICOS DIFERENTES
(UNO CON CANTIDAD PEDIDA Y OTRO CON TOTALES GENERADOS)

Con Totales Generados...

In [61]:
# Crear el objeto figura
fig = go.Figure()

for i in range(12):
    fig.add_trace(go.Scatter(
        x=listaDias, y=listaValores[i],  # Estas dos líneas deben estar dentro de la misma llamada a go.Scatter
        mode='lines',
        name=listaMeses[i],
        line=dict(shape='spline', smoothing=1.3, width=2)
    ))

# Configurar los ejes y los títulos
fig.update_layout(
    title="Ventas por día según los Meses",
    title_x=0.5,
    xaxis_title="Día del Mes",
    yaxis_title="Ventas Diarias",
    legend=dict(x=1, y=0.5),

    xaxis=dict(title="Dia Del Mes",tickvals=list(listaDias), ticktext=listaDias),
    yaxis=dict(title="Totales Generados", side="left"))


fig.show()

Con Cantidad Pedida...

In [62]:
#para cantidades vendidas
patron = dataframe_con_direcciones.groupby(['Mes','Dia'])['Cantidad Pedida'].sum().unstack().reindex(meses)
#Convertimos en listas para poder graficar luego
listaDiasCantidad = list(patron.columns)
listaMesesCantidad = list(patron.index)
listaValoresCantidad = list(patron.values)


In [63]:
# Crear el objeto figura
fig = go.Figure()

for i in range(12):
    fig.add_trace(go.Scatter(
        x=listaDiasCantidad, y=listaValoresCantidad[i],  # Estas dos líneas deben estar dentro de la misma llamada a go.Scatter
        mode='lines',
        name=listaMesesCantidad[i],
        line=dict(shape='spline', smoothing=1.3, width=2)
    ))

# Configurar los ejes y los títulos
fig.update_layout(
    title="Cantidades Pedidas por día según los Meses",
    title_x=0.5,
    xaxis_title="Día del Mes",
    yaxis_title="Cantidades pedidas Diarias",
    legend=dict(x=1, y=0.5),

    xaxis=dict(title="Dia Del Mes",tickvals=list(listaDiasCantidad), ticktext=listaDiasCantidad),
    yaxis=dict(title="Cantidades pedidas", side="left"))


fig.show()


¿Cómo varían las ventas a lo largo de los diferentes días de la semana?

In [65]:
#Creamos una lista con los dias de la semana para poder ordenar luego
listaSemanaOrdenada = ["Monday","Tuesday", "Wednesday","Thursday","Friday","Saturday","Sunday"]


#Creamos la funcion para agregar una columna para diferencias que dia de la semana es (datetime)
def agregarDiaDeSemana(datos):
  datos['Dia de la Semana'] = datos['Fecha de Pedido'].dt.strftime('%A')
  return datos

#Creamos un df con la columna del dia agregado
dataframe_con_dias = agregarDiaDeSemana(dataframe_completo)

#Agrupamos y ordenamos
totalVentasXdiaSemana = dataframe_con_dias.groupby('Dia de la Semana')['Total Generado'].sum()
semanaOrdenadaFinal = totalVentasXdiaSemana.reindex(listaSemanaOrdenada)



Creamos la figura para mostrar las ventas por dia de la semana.

In [66]:
# Crear el objeto figura
fig = go.Figure()

# Agregar un gráfico de barras con los datos de ventas por día de la semana
fig.add_trace(go.Bar(
    x=semanaOrdenadaFinal.index,
    y=semanaOrdenadaFinal.values,
    text=semanaOrdenadaFinal.values,  # Agregar etiquetas de texto con los valores
    textposition='auto',  # Colocar las etiquetas de texto automáticamente
    marker = dict(color= semanaOrdenadaFinal, colorscale = 'RdYlGn',colorbar = dict(title = "Ventas por Día de la Semana")),
))

# Configurar los ejes y el título del gráfico
fig.update_layout(
    title="Ventas por Día de la Semana",
    title_x=0.5,
    xaxis_title="Día de la Semana",
    yaxis_title="Total de Ventas",
)

# Mostrar el gráfico
fig.show()

¿Se observa alguna diferencia significativa en las ventas entre los días laborables y
los fines de semana?


**(Contestado en el informe)**

Impacto de eventos especiales en las ventas:

¿Se ha observado algún aumento o disminución significativa en las ventas en díascercanos a eventos especiales, como días festivos?
**(Contestado en el informe)**

(Utilizamos un grafico para RESALTAR EL MES DE DICIEMBRE)

y sacar conclusiones con navidad por ejemplo.

In [76]:
# Crear el objeto figura
fig = go.Figure()

for i in range(12):
    fig.add_trace(go.Scatter(
        x=listaDiasCantidad, y=listaValoresCantidad[i],  # Estas dos líneas deben estar dentro de la misma llamada a go.Scatter
        mode='lines',
        name=listaMesesCantidad[i],
        line=dict(shape='spline', smoothing=1.3, width=2)
    ))

# Configurar los ejes y los títulos
fig.update_layout(
    title="Cantidades Pedidas por día según los Meses",
    title_x=0.5,
    xaxis_title="Día del Mes",
    yaxis_title="Cantidades pedidas Diarias",
    legend=dict(x=1, y=0.5),

    xaxis=dict(title="Dia Del Mes",tickvals=list(listaDiasCantidad), ticktext=listaDiasCantidad),
    yaxis=dict(title="Cantidades pedidas", side="left"))


fig.show()

¿Qué eventos específicos han tenido un impacto notable en el comportamiento de
las ventas y cómo se manifestó ese impacto?

**(Contestado en el informe)**

(Utilizamos un grafico para RESALTAR EL MES DE NOVIEMBRE)

**Sacamos conclusiones por ejemplo tomando en cuenta LA RETIRADA DE EEUU del acuerdo de Paris sobre el cambio Climatico.**

(4 de Noviembre)

In [77]:
# Crear el objeto figura
fig = go.Figure()

for i in range(12):
    fig.add_trace(go.Scatter(
        x=listaDiasCantidad, y=listaValoresCantidad[i],  # Estas dos líneas deben estar dentro de la misma llamada a go.Scatter
        mode='lines',
        name=listaMesesCantidad[i],
        line=dict(shape='spline', smoothing=1.3, width=2)
    ))

# Configurar los ejes y los títulos
fig.update_layout(
    title="Cantidades Pedidas por día según los Meses",
    title_x=0.5,
    xaxis_title="Día del Mes",
    yaxis_title="Cantidades pedidas Diarias",
    legend=dict(x=1, y=0.5),

    xaxis=dict(title="Dia Del Mes",tickvals=list(listaDiasCantidad), ticktext=listaDiasCantidad),
    yaxis=dict(title="Cantidades pedidas", side="left"))


fig.show()

A CONTINUACION DIFERENCIAMOS LAS CANTIDADES PEDIDAS Y LOS TOTALES GENERADOS

Para cada dia en especifico.
Esto me permite ver las cantidades y los totales generados para un dia especifico del anio.

(Por ejemplo 24 de diciembre.)

Los dias con los MAYORES TOTALES GENERADOS




In [86]:

def extraer_fecha(datos):
  datos['Fecha de Pedido'] = pd.to_datetime(dataframe_completo['Fecha de Pedido'])
  datos['Fecha'] = datos['Fecha de Pedido'].dt.date
  return datos

dataConFecha = extraer_fecha(dataframe_completo)

#por total generado
diasConMasVentas = dataframe_completo.groupby('Fecha')['Total Generado'].sum()


diasmas = diasConMasVentas.sort_values(ascending=False)
diasmasDF = pd.DataFrame(diasmas)
diasmasDF.head(10) # Que solo muestre los 10 primeros

,Total Generado
Fecha,
2019-12-04,166727.69
2019-12-16,162970.61
2019-12-10,162832.74
2019-12-20,160181.30
2019-12-17,157364.72
2019-12-29,156024.62
2019-12-05,155079.30
2019-12-21,154756.87
2019-12-18,154598.77


Los dias con los MAYORES CANTIDADES PEDIDAS

In [ ]:
#por cantidad vendida
diasConMasVentasCantidad = dataframe_completo.groupby('Fecha')['Cantidad Pedida'].sum()

diasmasCantidad = diasConMasVentasCantidad.sort_values(ascending=False)
diasmasCantidadDF = pd.DataFrame(diasmasCantidad)
diasmasCantidadDF.head(10) # Que solo muestre los 10 primeros

PREGUNTA ADICIONAL

-- Una pregunta que se nos ocurrió fue si la franja horaria laboral promedio influía en las compras que se realizaron. Con el grafico realizado de ventas totales por hora podemos apreciar que el horario de las 19 hs es donde se producen la mayor cantidad de compras, correlacionándose con la franja laboral promedio de los estadounidenses (9 a 17) teniendo en cuenta que las personas tienen que recorrer una distancia que recorrer hasta su puesto de trabajo.

In [90]:
ventasXhora_por_mes = dataframe_completo.groupby(['Mes', 'Hora'])['Cantidad Pedida'].sum()

Horas= (list(range(24)))
# Crear el objeto figura
fig = go.Figure()

for mes in meses:
    ventas_mes_actual = ventasXhora_por_mes[mes]

    # Agregar al objeto figura un gráfico de línea con las horas y las ventas por hora para el mes actual
    fig.add_trace(go.Scatter(
        x=ventas_mes_actual.index,
        y=ventas_mes_actual.values,
        mode='lines',
        name=mes,
        line=dict(shape='spline', smoothing=1.3, width=2)))

# Configurar los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora",
    xaxis=dict(title="Horas",tickvals=Horas),
    yaxis=dict(title="Unidades vendidas", side="left"))

fig.show()